In [3]:
import os
import numpy as np
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import joblib
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Function to load and preprocess images using InceptionResNetV2
def load_and_preprocess_images(image_paths):
    img_list = []
    for img_path in image_paths:
        try:
            img = image.load_img(img_path, target_size=(160, 160))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)
            img_list.append(img_array)
        except Exception as e:
            print(f"Error processing image {img_path}: {e}")

    if len(img_list) > 0:
        return np.vstack(img_list)
    else:
        return None

# Function to load a dataset
def load_dataset(dataset_path):
    images = []
    labels = []

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.endswith(".jpg"):
                img_path = os.path.join(root, file)
                label = root.split("/")[-1]
                labels.append(label)
                images.append(img_path)

    return images, labels

# Replace this with your actual dataset path
dataset_path = "C:/Users/Admin/Documents/MTechProject/SAMFR/Faces"

# Load dataset
images, labels = load_dataset(dataset_path)

# Load InceptionResNetV2 as FaceNet model
facenet_model = InceptionResNetV2(weights='imagenet', include_top=False)

# Extract features using FaceNet
X = load_and_preprocess_images(images)
if X is not None:
    features = facenet_model.predict(X)

    # Encode labels using scikit-learn LabelEncoder
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.35, random_state=80)

    # Train a KNN classifier on the training set with fewer neighbors
    knn_classifier = KNeighborsClassifier(n_neighbors=7)  # Adjust this value
    knn_classifier.fit(X_train.reshape(len(X_train), -1), y_train)

    # Save the KNN classifier
    joblib.dump(knn_classifier, "C:/Users/Admin/Documents/MTechProject/SAMFR/knn_model.pkl")

    # Save the FaceNet model in the recommended Keras format
    save_model(facenet_model, "C:/Users/Admin/Documents/MTechProject/SAMFR/facenet_model.keras")

    # Save the label encoder
    joblib.dump(label_encoder, "C:/Users/Admin/Documents/MTechProject/SAMFR/label_encoder.pkl")

    # Use the trained model to make predictions on the testing set
    y_pred = knn_classifier.predict(X_test.reshape(len(X_test), -1))

    # Decode the labels
    y_test_decoded = label_encoder.inverse_transform(y_test)
    y_pred_decoded = label_encoder.inverse_transform(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_test_decoded, y_pred_decoded)

    print(f"Accuracy: {accuracy * 100:.2f}%")
else:
    print("No valid images found.")


81/81 [==============================] - 108s 1s/step
Accuracy: 100.00%
